In [ ]:
# Autoreload
%cd ..
%load_ext autoreload
%autoreload 2

In [ ]:
# Imports
from tools import get_client, read_yaml, read_csv
from tqdm import tqdm

In [ ]:
# Load AMT client
config = read_yaml('config.yaml')
aws_keys = read_yaml('aws_keys.yaml')

client = get_client(aws_keys, config)

In [ ]:
# Approve assignments
in_file = 'data/ans_data.csv'
assignment_id_field = 'AssignmentId'

cont = read_csv(in_file)

# Optionally filter rows with condition
approval_condition = lambda row: row['reject'] == 'No'
approved_rows = list(filter(approval_condition, cont))

for row in tqdm(approved_rows, ascii=True):
    aid = row[assignment_id_field]
    try:
        client.approve_assignment(AssignmentId=aid, OverrideRejection=True)
    except client.exceptions.RequestError:
        # Already Approved
        pass

In [ ]:
# Reject assignments
in_file = 'data/ans_data.csv'
assignment_id_field = 'AssignmentId'
answer_field = 'DescriptionText'

# It is good to give an adequate reason for rejections
rejection_message = 'Your submission ("{answer}") did not follow the instructions because ...'

cont = read_csv(in_file)

# Optionally filter rows with condition
approval_condition = lambda row: row['reject'] == 'Yes'
rejected_rows = list(filter(rejection_condition, cont))

for row in tqdm(rejected_rows, ascii=True):
    aid = row[assignment_id_field]
    msg = rejection_message.format(answer=row[answer_field])
    try:
        client.reject_assignment(AssignmentId=aid, RequesterFeedback=rejection_message)
    except client.exceptions.RequestError:
        # Already rejected or approved
        pass